# 2 Azure Maps Geocoding and Reverse Geocoding

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import datetime
import folium
import os
import pandas as pd
import requests
import sys

from dotenv import load_dotenv
from IPython.display import IFrame, FileLink
from typing import List, Dict, Tuple

## Settings

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 01-Sep-2025 15:04:15


In [4]:
load_dotenv("azure.env")

True

In [5]:
RESULTS_DIR = "results"

os.makedirs(RESULTS_DIR, exist_ok=True)

In [6]:
class AzureMapsClient:
    """
    Azure Maps API Client for Python
    """
    def __init__(self, subscription_key: str):
        """
        Initialize Azure Maps client
        
        Args:
            subscription_key: Your Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com"

    def _make_request(self, endpoint: str, params: Dict) -> Dict:
        """
        Make authenticated request to Azure Maps API
        
        Args:
            endpoint: API endpoint
            params: Request parameters
            
        Returns:
            API response as dictionary
        """
        params['api-version'] = '1.0'
        params['subscription-key'] = self.subscription_key

        url = f"{self.base_url}/{endpoint}"

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return {}

## Helper

In [7]:
class GeocodingDemo:
    """
    Demonstrates Azure Maps geocoding capabilities
    """
    def __init__(self, azure_maps_client):
        self.client = azure_maps_client

    def geocode_address(self, address: str, country_set: str = None) -> Dict:
        """
        Convert address to coordinates (geocoding)
        
        Args:
            address: Street address to geocode
            country_set: ISO 3166-1 country codes to limit search (e.g., "US,CA")
            
        Returns:
            Geocoding result
        """
        params = {'query': address, 'limit': 5}

        if country_set:
            params['countrySet'] = country_set

        response = self.client._make_request('search/address/json', params)
        return response

    def reverse_geocode(self, latitude: float, longitude: float) -> Dict:
        """
        Convert coordinates to address (reverse geocoding)
        
        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate
            
        Returns:
            Reverse geocoding result
        """
        params = {'query': f"{latitude},{longitude}"}

        response = self.client._make_request('search/address/reverse/json',
                                             params)
        return response

    def batch_geocode(self, addresses: List[str]) -> List[Dict]:
        """
        Geocode multiple addresses
        
        Args:
            addresses: List of addresses to geocode
            
        Returns:
            List of geocoding results
        """
        results = []
        for address in addresses:
            result = self.geocode_address(address)
            results.append(result)
        return results

In [8]:
def print_json(data: Dict, indent: int = 2):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=indent, ensure_ascii=False))


def extract_coordinates(result: Dict) -> Tuple[float, float]:
    """
    Extract latitude and longitude from Azure Maps result
    
    Args:
        result: Azure Maps API result object
        
    Returns:
        Tuple of (latitude, longitude)
    """
    if 'position' in result:
        pos = result['position']
        return pos['lat'], pos['lon']
    return None, None

## Examples

In [9]:
azure_maps = AzureMapsClient(os.getenv('AZURE_MAPS_KEY'))

In [10]:
geocoding = GeocodingDemo(azure_maps)

### Example 1: Basic geocoding

In [11]:
address = "1 Northeast One Microsoft Way, Redmond, WA 98052, USA"

result = geocoding.geocode_address(address)

if result and 'results' in result:
    for i, res in enumerate(result['results'][:3]):  # Show top 3 results
        lat, lon = extract_coordinates(res)
        confidence = res.get('score', 'N/A')
        formatted_address = res.get('address', {}).get('freeformAddress',
                                                       'N/A')

        print(f"Result {i+1}:")
        print(f"  Address: {formatted_address}")
        print(f"  Coordinates: {lat}, {lon}")
        print(f"  Confidence: {confidence}")
        print()

Result 1:
  Address: 1 Northeast One Microsoft Way, Redmond, WA 98052
  Coordinates: 47.641673, -122.125648
  Confidence: 1



### Example 2: Reverse geocoding

In [12]:
if result and 'results' in result and result['results']:
    first_result = result['results'][0]
    lat, lon = extract_coordinates(first_result)

    reverse_result = geocoding.reverse_geocode(lat, lon)

    if reverse_result and 'addresses' in reverse_result:
        address_info = reverse_result['addresses'][0]['address']
        print(f"Coordinates: {lat}, {lon}")
        print(f"Reverse geocoded address:")
        print(f"  Street: {address_info.get('streetName', 'N/A')} {address_info.get('streetNumber', '')}")
        print(f"  City: {address_info.get('municipality', 'N/A')}")
        print(f"  State: {address_info.get('countrySubdivision', 'N/A')}")
        print(f"  Country: {address_info.get('country', 'N/A')}")
        print(f"  Postal Code: {address_info.get('postalCode', 'N/A')}")

Coordinates: 47.641673, -122.125648
Reverse geocoded address:
  Street: Northeast One Microsoft Way 1
  City: Redmond
  State: WA
  Country: United States
  Postal Code: 98052


### Example 3: Batch geocoding

In [13]:
sample_addresses = [
    "Paris, France", "Lille, France", "Cabourg, France", "Nice, France",
    "Bordeaux, France", "Brighton, UK", "Firenze, Italie"
]

batch_results = geocoding.batch_geocode(sample_addresses)

# Create DataFrame for results
geocoding_data = []
for i, result in enumerate(batch_results):
    if result and 'results' in result and result['results']:
        first_result = result['results'][0]
        lat, lon = extract_coordinates(first_result)
        formatted_address = first_result.get('address',
                                             {}).get('freeformAddress', 'N/A')

        geocoding_data.append({
            'original_query': sample_addresses[i],
            'formatted_address': formatted_address,
            'latitude': lat,
            'longitude': lon,
            'confidence': first_result.get('score', 'N/A')
        })

df = pd.DataFrame(geocoding_data)
df

,original_query,formatted_address,latitude,longitude,confidence
0,"Paris, France",Paris,48.856895,2.350849,0.954208
1,"Lille, France",Lille,50.631286,3.062753,0.943229
2,"Cabourg, France",Cabourg,49.285925,-0.122589,1.000000
3,"Nice, France",Nice,43.703427,7.266266,1.000000
4,"Bordeaux, France",Bordeaux,44.834995,-0.575495,1.000000
5,"Brighton, UK",Brighton,50.819522,-0.136420,1.000000
6,"Firenze, Italie",Firenze,43.768698,11.256929,0.991207


### Example 4: Create map visualization

In [14]:
def create_geocoding_map(geocoding_results: pd.DataFrame,
                         tiles='OpenStreetMap',
                         center_coords: Tuple[float, float] = None):
    """
    Create folium map with geocoding results
    
    Args:
        geocoding_results: DataFrame with geocoding results
        center_coords: Optional center coordinates for map
        
    Returns:
        Folium map object
    """
    if center_coords is None:
        # Calculate center from all points
        center_lat = geocoding_results['latitude'].mean()
        center_lon = geocoding_results['longitude'].mean()
    else:
        center_lat, center_lon = center_coords

    # Create map
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=5,
                   tiles=tiles)

    # Add markers for each geocoded location
    for _, row in geocoding_results.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=
            f"<b>{row['original_query']}</b><br>{row['formatted_address']}<br>Confidence: {row['confidence']}",
            tooltip=row['original_query'],
            icon=folium.Icon(color='red', icon='info-sign')).add_to(m)

    return m


In [15]:
geocoding_map = create_geocoding_map(df, tiles="OpenStreetMap", center_coords=(48.85, 2.35))

output_file = os.path.join(RESULTS_DIR, "azure_maps_geocoding_demo.html")
geocoding_map.save(output_file)
print(f"✅ Map saved as {output_file}")

✅ Map saved as results/azure_maps_geocoding_demo.html


In [16]:
IFrame(src=output_file, width=800, height=600)

In [17]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/azure_maps_geocoding_demo.html

In [18]:
geocoding_map = create_geocoding_map(df, tiles="CartoDB dark_matter", center_coords=(48.85, 2.35))

output_file = os.path.join(RESULTS_DIR, "azure_maps_geocoding_demo2.html")
geocoding_map.save(output_file)
print(f"✅ Map saved as {output_file}")

✅ Map saved as results/azure_maps_geocoding_demo2.html


In [19]:
IFrame(src=output_file, width=800, height=600)

In [20]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/azure_maps_geocoding_demo2.html

### Example 5: Advanced geocoding with filters

In [21]:
# Search for "Paris" with and without country filter
print("Searching for 'Paris' without filter:")
paris_results = geocoding.geocode_address("Paris")

if paris_results and 'results' in paris_results:
    for i, res in enumerate(paris_results['results'][:3]):
        country = res.get('address', {}).get('country', 'N/A')
        formatted_address = res.get('address', {}).get('freeformAddress',
                                                       'N/A')
        print(f"  {i+1}. {formatted_address} ({country})")

print("\nSearching for 'Paris' with France filter:")
paris_france = geocoding.geocode_address("Paris", country_set="FR")

if paris_france and 'results' in paris_france:
    for i, res in enumerate(paris_france['results'][:3]):
        country = res.get('address', {}).get('country', 'N/A')
        formatted_address = res.get('address', {}).get('freeformAddress',
                                                       'N/A')
        print(f"  {i+1}. {formatted_address} ({country})")

Searching for 'Paris' without filter:
  1. Paris (France)
  2. Paris, TX (United States)
  3. Paris, TN (United States)

Searching for 'Paris' with France filter:
  1. Paris (France)
  2. Paris-l'Hôpital (France)
  3. Le Touquet-Paris-Plage (France)
